<a href="https://colab.research.google.com/github/pe44enka/LoadBalancer/blob/main/LoadBalancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Load Balancer**

![](https://phoenixnap.com/kb/wp-content/uploads/2021/06/how-does-load-balancing-work.png)


### Objectives
Imagine you have a cloud and you have to manage it such a way that there are enough available servers to serve incoming connections. Load Balancer will help you to automate this proceess.

---

### Goal of the project
To create a Load Balancer that ensure that load of each server wouldn't exceed 50% and add more servers in other case.

### Technique
For this project we will use fundamental concepts of Object-Oriented Programming like classes and composition.

### Library
* **```random```**: used to create random amount of load to the server
* **```numpy.median```**: used to find median of total loads to each server

---

# Libraries

In [1]:
import random
from numpy import median

# Server Class

To represent the servers that are taking care of the connections, we will use a *Server class*: 
* each connection is represented by an id, e.g. ip address of the computer connecting to the server
* for our simulation, each connection creates a random amount of load in the server, between 1% and 10% of server capacity
* also create methods associated with this class:
  * ```add_connection```: add a new connection to the server 
  * ```close_connection```: close connection on the server
  * ```load```: calculate the current load from all connections to the server
  * ```__str__```: define how an instance of Server class will be printed

---

In [2]:
class Server:
    def __init__(self):
        """Creates a new server instance, with no active connections."""
        self.connections = {}

    def add_connection(self, connection_id):
        """Adds a new connection to this server."""
        connection_load = random.random()*9+1
        # Add the connection to the dictionary with the calculated load
        self.connections[connection_id] = connection_load

    def close_connection(self, connection_id):
        """Closes a connection on this server."""
        # Remove the connection from the dictionary
        del(self.connections[connection_id])

    def load(self):
        """Calculates the current load for all connections."""
        total = 0
        # Add up the load for each of the connections
        for load in self.connections.values():
            total += load
        return total

    def __str__(self):
        """Returns a string with the current load of the server"""
        return "{:.2f}%".format(self.load())

---

Let's create a Server class instance and check how it works.

---

In [3]:
server = Server()
server.add_connection("192.168.1.1")

print(server)

9.59%


---

Now let's try to remove connection.

---

In [4]:
server.close_connection("192.168.1.1")

print(server)

0.00%




---

As long as we had just one connection to particular server the load dropped to zero.

---

# LoadBalancer Class

Now let's create *LoadBalancer* class that takes *Server* class instances as attributes. This type of relationship is known as *Composition*.

This class:
* will start with only one server available
* when a connection is added -> it will randomly select a server to serve that connection -> pass on the connection to the server
* also needs to keep track of the ongoing connections to be able to close them
* includes following methods:
  * ```add_connection```: add connection to randomly chosen server
  * ```close_connection```: close connection with particular id
  * ```avg_load```: calculate average load in the system (of all servers together)
  * ```__str__```: define how an instance of LoadBouncer class will be printed

---


In [5]:
class LoadBalancer:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Randomly selects a server and adds a connection to it."""
        server = random.choice(self.servers)
        # Add the connection to the dictionary with the selected server
        self.connections[connection_id] = server
        # Add the connection to the server
        server.add_connection(connection_id)

    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        # Find out the right server
        server = self.connections[connection_id]
        # Close the connection on the server
        server.close_connection(connection_id)
        # Remove the connection from the load balancer
        del(self.connections[connection_id])

    def avg_load(self):
        """Calculates the average load of all servers"""
        # Sum the load of each server and divide by the amount of servers
        load_sum = 0
        for server in self.servers:
            load_sum += server.load() 
        load_avg = load_sum/len(self.servers)
        return load_avg

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(server) for server in self.servers]
        load_avg = self.avg_load()
        return "Average load of the servers is {:.2f}%\nLoads of each server: [{}]".format(load_avg, (",".join(loads)))

---

Let's create a LoadBalancer class instance and check how it works.

---

In [6]:
loadbalancer = LoadBalancer()
print(loadbalancer)

Average load of the servers is 0.00%
Loads of each server: [0.00%]


---

As long as there is no connection added, load of the server is 0%. Let's add connection to see how load will change.

---

In [7]:
loadbalancer.add_connection("192.168.1.1") #add connection to the server
print(loadbalancer)

Average load of the servers is 5.52%
Loads of each server: [5.52%]


---

Now when we have a load to the server, let's add another server to the system to see how load will change in this case.

---

In [8]:
loadbalancer.servers.append(Server()) # add 1 more server to system
print(loadbalancer)

Average load of the servers is 2.76%
Loads of each server: [5.52%,0.00%]




---

Average load decreases twice when we added another server. Let's automete this process by adding a new method to the class:
```ensure_availability```: check if the average load exceeds 50% and add as many servers as needed to keep it under 50%.

---

In [9]:
class LoadBalancer:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Randomly selects a server and adds a connection to it."""
        server = random.choice(self.servers)
        # Add the connection to the dictionary with the selected server
        self.connections[connection_id] = server
        # Add the connection to the server
        server.add_connection(connection_id)

    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        # Find out the right server
        server = self.connections[connection_id]
        # Close the connection on the server
        server.close_connection(connection_id)
        # Remove the connection from the load balancer
        del(self.connections[connection_id])

    def avg_load(self):
        """Calculates the average load of all servers"""
        # Sum the load of each server and divide by the amount of servers
        load_sum = 0
        for server in self.servers:
            load_sum += server.load() 
        load_avg = load_sum/len(self.servers)
        return load_avg

    def ensure_availability(self):
        """If the average load is higher than 50, spin up a new server"""
        while self.avg_load()>50:
            self.servers.append(Server())

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(round(server.load(),2))+'%' for server in self.servers]
        load_avg = self.avg_load()
        return "Average load of the servers is {:.2f}%\nLoads of each server: [{}]".format(load_avg, (",".join(loads)))

In [10]:
loadbalancer = LoadBalancer()

In [11]:
for connection in range(20):
    loadbalancer.add_connection(connection)
loadbalancer.ensure_availability()
print(loadbalancer)

Average load of the servers is 47.86%
Loads of each server: [95.72%,0%]




---

As we can see adding new servers to the system helps with decreasing the average load, but it doesn't keep load of each server under 50%. To see it more clearly lets add ```median_load``` method: show *median* value of server loads.

---



In [12]:
class LoadBalancer:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Randomly selects a server and adds a connection to it."""
        server = random.choice(self.servers)
        # Add the connection to the dictionary with the selected server
        self.connections[connection_id] = server
        # Add the connection to the server
        server.add_connection(connection_id)

    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        # Find out the right server
        server = self.connections[connection_id]
        # Close the connection on the server
        server.close_connection(connection_id)
        # Remove the connection from the load balancer
        del(self.connections[connection_id])

    def avg_load(self):
        """Calculates the average load of all servers"""
        # Sum the load of each server and divide by the amount of servers
        load_sum = 0
        for server in self.servers:
            load_sum += server.load() 
        load_avg = load_sum/len(self.servers)
        return load_avg
    
    def median_load(self):
        """Calculates the median load of all servers"""
        # create list of server loads
        load_list = [server.load() for server in self.servers]
        # find median of the server loads list
        load_med = median(load_list)
        return load_med
    
    def ensure_availability(self):
        """If the average load is higher than 50, spin up a new server"""
        while self.avg_load()>50:
            self.servers.append(Server())

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(round(server.load(),2))+'%' for server in self.servers]
        load_avg = self.avg_load()
        load_med = self.median_load()
        return "Average load of the servers is {:.2f}%\nMedian load of servers: {:.2f}%\nLoads of each server: [{}]".format(load_avg, load_med, (",".join(loads)))

In [13]:
loadbalancer= LoadBalancer()

In [14]:
for connection in range(50):
    loadbalancer.add_connection(connection)
loadbalancer.ensure_availability()
print('{} servers used with {:.2f}% average load'.format(len(loadbalancer.servers),loadbalancer.avg_load()))

6 servers used with 44.67% average load


In [15]:
print(loadbalancer)

Average load of the servers is 44.67%
Median load of servers: 0.00%
Loads of each server: [268.01%,0%,0%,0%,0%,0%]




---

Median clearly shows the bias in load distribution. To avoid this we will disclaim the rule of random selection of the server when new connection appears. Instead, we will check if particular server is loaded enough (40% or more) or not:
* incase the load of the server is less than 40% -> it can server at least one more connection (which load is randomly choosen from 1% to 10%)
* in case the load of the server more than 40% -> it can't server any more connections
* in case there is no more servers available for loading (all servers have loads more than 40%) -> we add a new server to the system and add load from a new connection to it

In this case when we check availability of servers at the moment of adding  connections, we don't need ```ensure_availability``` method anymore.

---



In [16]:
class LoadBalancer:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Selects a server based on its availability and adds a connection to it."""
        server = '' 
        #create a list of each server loads
        server_load = [server.load() for server in self.servers]
        #check if load on particular server < 40% and choose this server in case it is
        for load in server_load:
          if load <= 40: 
            server = self.servers[server_load.index(load)]
        #add a new server to the system in case there is no available servers
        if not server:
          self.servers.append(Server())
          server = self.servers[-1] 
        # Add the connection to the dictionary with the selected server
        self.connections[connection_id] = server
        # Add the connection to the server
        server.add_connection(connection_id)

    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        # Find out the right server
        server = self.connections[connection_id]
        # Close the connection on the server
        server.close_connection(connection_id)
        # Remove the connection from the load balancer
        del(self.connections[connection_id])

    def avg_load(self):
        """Calculates the average load of all servers"""
        # Sum the load of each server and divide by the amount of servers
        load_sum = 0
        for server in self.servers:
            load_sum += server.load() 
        load_avg = load_sum/len(self.servers)
        return load_avg
    
    def median_load(self):
        """Calculates the median load of all servers"""
        # create list of server loads
        load_list = [server.load() for server in self.servers]
        # find median of the server loads list
        load_med = median(load_list)
        return load_med

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(server) for server in self.servers]
        load_avg = self.avg_load()
        load_med = self.median_load()
        return "Average load of the servers is {:.2f}%\nMedian load of servers: {:.2f}%\nLoads of each server: [{}]".format(load_avg, load_med, (",".join(loads)))

In [17]:
loadbalancer= LoadBalancer()

In [18]:
for connection in range(20):
    loadbalancer.add_connection(connection)
print('{} servers used with {:.2f}% average load'.format(len(loadbalancer.servers),loadbalancer.avg_load()))

3 servers used with 31.15% average load


In [19]:
print(loadbalancer)

Average load of the servers is 31.15%
Median load of servers: 40.82%
Loads of each server: [40.82%,46.14%,6.50%]




---

And finally we found a way to use as few servers as possible and keep their loads less than 50%. Such a way we balanced load distribution among servers.

---

